# Dashboard

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

In [1]:
import warnings
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.io import output_notebook, show
from bokeh.models import (
    GeoJSONDataSource,
    Slider,
    CustomJS,
    LinearColorMapper,
    Range1d,
    ColumnDataSource,
    LabelSet,
    Span
)
from bokeh.models import HoverTool
from bokeh.plotting import figure
from bokeh.layouts import column, row
import json
from bokeh.models import LinearAxis
from bokeh.palettes import PuBu, Greens, Reds
from IPython.display import display
from IPython.display import HTML

%pip install ipywidgets
%pip install colorcet
import colorcet
import ipywidgets as widgets



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load world geometry data
world = gpd.read_file("data/ne_110m_admin_0_countries.shp")

AFR = ["Algeria","Angola","Benin","Botswana,Burkina Faso","Burundi","Cameroon","Congo","Côte d’Ivoire","Democratic Republic of the Congo","Djibouti","Egypt","Eritrea","Ethiopia","Gabon","The Gambia","Ghana","Kenya","Libya","Madagascar","Malawi","Mali","Mauritania","Mauritius","Morocco","Namibia","Nigeria","Rwanda","Senegal","Sierra Leone","South Africa","Somalia","Sudan","Togo","Tunisia","Uganda","Zambia",]
APAC = ["Afghanistan","Azerbaijan","Bahrain","Bangladesh","China","Fiji","India","Indonesia","Iraq","Japan","Jordan","Kazakhstan","Kuwait","Kyrgyzstan","Malaysia","Maldives","Marshall Islands","Mongolia","Nepal","Pakistan","Philippines","Qatar","Republic of Korea","Saudi Arabia","Sri Lanka","Thailand","United Arab Emirates","Uzbekistan","Viet Nam",]
GRULAC = ["Argentina","Bahamas","Bolivia (Plurinational State of)","Brazil","Chile","Costa Rica","Cuba","Ecuador","El Salvador","Guatemala","Honduras","Mexico","Nicaragua","Panama","Paraguay","Peru","Uruguay","Venezuela (Bolivarian Republic of)",]
WEOG = ["Australia","Austria","Belgium","Canada","Denmark","Finland","France","Germany","Iceland","Ireland","Italy","Luxembourg","Netherlands","Norway","Portugal","Spain","Switzerland","United Kingdom of Great Britain and Northern Ireland","United States of America",]
EG = ["Albania","Armenia","Bosnia and Herzegovina","Bulgaria","Czechia","Croatia","Estonia","Georgia","Hungary","Latvia","Lithuania","Montenegro","Poland","Republic of Moldova","Republic of North Macedonia","Romania","Russian Federation","Slovakia","Slovenia","Ukraine",]

# Function to assign color based on country group
def assign_color(country):
    if country in AFR:
        return "blue"
    elif country in APAC:
        return "green"
    elif country in GRULAC:
        return "red"
    elif country in WEOG:
        return "yellow"
    elif country in EG:
        return "orange"
    else:
        return "lightgray"


def get_country_group(country):
    if country in AFR:
        return "AFR"
    elif country in APAC:
        return "APAC"
    elif country in GRULAC:
        return "GRULAC"
    elif country in WEOG:
        return "WEOG"
    elif country in EG:
        return "EG"


def calculate_alignment(row):
    if row["Country"] == "China":
        return 1
    if row["China Vote"] == "Abstaining":
        return 0
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1


def add_alignment_percentage(df):
    try:
        df["alignment_percentage"] = (
            df["alignment_score"] / df["Number of votes"]
        ) * 100
    except Exception:
        pass
    return df


def add_alignment(df):
    try:
        df["alignment"] = df["alignment_score"] / df["Number of votes"]
    except Exception:
        pass
    return df

def add_alignment_change(df):
    try:
        df["alignment_change"] = df["alignment"] - df["alignment"].shift(1)
    except Exception:
        pass
    return df

def add_fa_change(df):
    try:
        df["FT_ODI_change"] = df["FT_ODI"] - df["FT_ODI"].shift(1)
        df["AEI_FDI_change"] = df["AEI_FDI"] - df["AEI_FDI"].shift(1)
        df["BU_ODA_change"] = df["BU_ODA"] - df["BU_ODA"].shift(1)
        df["ODI_MOFCOM_change"] = df["ODI_MOFCOM"] - df["ODI_MOFCOM"].shift(1)
    except Exception:
        pass
    return df

def add_average_fa(df):
    try:
        df["Average_FA"] = (
            df[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]]
            .replace(0, np.nan)
            .mean(axis=1, skipna=True)
        )
        df["Average_FA"] = df["Average_FA"].fillna(0)
    except Exception:
        pass
    return df

def add_average_fa_change(df):
    try:
        df["Average_FA_change"] = df["Average_FA"] - df["Average_FA"].shift(1)
    except Exception:
        pass
    return df

def add_measures(df):
    df = add_alignment_percentage(df)
    df = add_alignment(df)
    df = add_average_fa(df)
    return df

def filter_country_group(row, country_group):
    return get_country_group(row["Country"]) == country_group


# Load and clean data:

# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)
df_vac = pd.read_csv("data/vac.csv")


# Clean vaccine data
df_vac.fillna(0, inplace=True)
df_vac.rename(
    columns={
        "Destination": "Country",
        "Entry Date (date of delivery)": "Year",
        "Arrived (D)": "Quantity",
    },
    inplace=True,
)
df_vac = df_vac.drop(columns=["Region"])
df_vac["Quantity"] = df_vac["Quantity"].replace("-", np.nan)
df_vac.dropna(axis=0, subset=["Year", "Quantity"], inplace=True)
df_vac = df_vac[df_vac["Quantity"] != 0]
df_vac["Year"] = df_vac["Year"].replace("Aug", "01-Aug-21")
df_vac["Year"] = df_vac["Year"].replace("Oct-21", "01-Oct-21")
df_vac["Year"] = df_vac["Year"].replace("17 May 21", "17-May-21")
df_vac.head()
df_vac["Year"] = pd.to_datetime(df_vac["Year"], format="%d-%b-%y", dayfirst=True).dt.year
df_vac = df_vac.astype({"Year": "int", "Quantity": "float32"})
df_vac = df_vac.groupby(["Year", "Country"]).sum().reset_index()

# Clean vote data
df["Country Group"] = df["Country"].apply(get_country_group)
china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)

df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_votes = (
    df.groupby(["Year", "Country", "Country Group", "Topic"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
        }
    )
    .reset_index()
)
df_votes.rename({"Text title": "Number of votes"}, axis=1, inplace=True)

# Clean FA data
df_fa = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_fa[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]] = df_fa[
    ["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]
].fillna(0)


# Global widgets
def get_filter(name):
    match name:
        case "fa_type":
            return widgets.Dropdown(
                options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average_FA"],value="FT_ODI",
                description="FA Type:",
            )
        case "country_group":
            return widgets.Dropdown(
                options=["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"], description="Country Group:", value="All"
            )
        case "country":
            return widgets.Dropdown(options=df.sort_values("Country")["Country"].unique(), description="Country:", value=df["Country"].unique().tolist()[0])
        case "topic":
            return widgets.Dropdown(
                options=["All"]+ df["Topic"].unique().tolist() ,
                description="Topic:",value="All"
            )

In [3]:
# Comparison of country voting alignment with China and Financial Assistance
# over time for all countries, absolute number of votes cast

# Years and countries to normalize data
years = pd.DataFrame({"Year": df["Year"].unique()})
countries = pd.DataFrame({"Country": world["ADMIN"].unique()})
df_map_fa = df_fa
df_map_fa = add_measures(df_map_fa)
data_map = df_votes
data_map = (
    data_map.groupby(["Year", "Country"])
    .agg({"alignment_score": "sum", "Number of votes": "sum"})
    .reset_index()
)
data_map = add_measures(data_map)
data_map = data_map.merge(df_fa, on=["Year", "Country"], how="left")
cya = years.merge(countries, how="cross")
cya = cya.merge(data_map, on=["Year", "Country"], how="left")
# Merge your aggregated data with the geopandas DataFrame
merged = world.merge(cya, left_on="ADMIN", right_on="Country", how="left")
merged = merged[
    [
        "featurecla",
        "CONTINENT",
        "REGION_UN",
        "ADMIN",
        "SUBREGION",
        "REGION_WB",
        "MIN_ZOOM",
        "MIN_LABEL",
        "MAX_LABEL",
        "LABEL_X",
        "LABEL_Y",
        "geometry",
        "Year",
        "Country",
        "alignment_score",
        "Number of votes",
        "alignment",
        "alignment_percentage",
        "FT_ODI",
        "AEI_FDI",
        "BU_ODA",
        "ODI_MOFCOM",
        "Average_FA",
    ]
]

merged_initial = merged[(merged["Year"] == 2006)]
# Convert to GeoJSON
geojson = json.dumps(merged_initial.__geo_interface__)
geojson_original = json.dumps(merged.__geo_interface__)
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson=geojson)

# Determine the bounds for your map
x_range = Range1d(start=-180, end=180)
y_range = Range1d(start=-90, end=90)


def render_layout(fa_type):
    tools = "wheel_zoom,pan,reset"
    va_map = figure(
        title="Voting Alignment with China",
        height=400,
        width=650,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )

    # Define color mapper
    color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38, low=100, high=-100
    )

    # Add patch renderer to figure
    va_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": "alignment_percentage", "transform": color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
    ]

    va_map.add_tools(HoverTool(tooltips=tooltips))

    slider = Slider(
        start=df["Year"].min(),
        end=df["Year"].max(),
        value=df["Year"].min(),
        step=1,
        title="Year",
    )
    callback = CustomJS(
        args=dict(source=geosource, available=geojson_original, slider=slider),
        code="""
        const year = slider.value;
        const geojson = (JSON.parse(source.geojson))
        const features = JSON.parse(available).features.filter(feature => feature.properties.Year === year);
        const new_geojson = {type:geojson.type, features:features}

        source.geojson = JSON.stringify(new_geojson);
        source.change.emit();
    """,
    )
    slider.js_on_change("value", callback)
    fa_color_mapper = LinearColorMapper(
        palette=colorcet.b_diverging_gwr_55_95_c38,
        low=0,
        high=df_fa[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]].max(axis=1).max(),
    )
    fa_map = figure(
        title="Financial Assistance from China",
        height=400,
        width=650,
        x_range=x_range,
        y_range=y_range,
        tools=tools,
    )
    fa_map.patches(
        "xs",
        "ys",
        source=geosource,
        fill_color={"field": fa_type, "transform": fa_color_mapper},
        line_color="black",
        line_width=0.5,
    )
    tooltips = [
        ("Country", "@Country"),
        ("Alignment Percent", "@alignment"),
        ("Financial Assistance", f"@{fa_type}" + "{int}"),
        ("Financial Assistance type", fa_type),
    ]
    fa_map.add_tools(HoverTool(tooltips=tooltips))
    layout = column(slider, row(va_map, fa_map))
    show(layout)

fa_type = get_filter("fa_type")
plot1 = widgets.interactive(
    render_layout,
    fa_type=fa_type,
)

In [4]:
# Country voting alignment % vs FA by FA type
# over time for all countries

fa = df_fa
votes = df_votes

def plot(fa_type, country_group, topic):
    plot = None
    group_by = {"Year", "Country Group", "Topic"}
    filtered_votes = df_votes
    filtered_fa = fa
    years = pd.DataFrame({"Year": df["Year"].unique()})

    if country_group == "All":
        group_by.remove("Country Group")
    else:
        filtered_votes = df_votes[df_votes["Country Group"]== country_group]
        mask = filtered_fa.apply(filter_country_group, country_group=country_group, axis=1)
        filtered_fa = filtered_fa[mask]
        
    if topic == "All":
        group_by.remove("Topic")
    else:
        filtered_votes = filtered_votes[filtered_votes["Topic"] == topic]
    
    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_fa = filtered_fa.groupby(list(set(filtered_fa.columns) & group_by)).agg(
        {
            "FT_ODI": "sum",
            "AEI_FDI": "sum",
            "BU_ODA": "sum",
            "ODI_MOFCOM": "sum",
        }
    ).reset_index()
    filtered_fa = add_measures(filtered_fa)
    filtered_votes = add_measures(filtered_votes)
    filtered_fa = years.merge(filtered_fa, on=["Year"], how="left")
    filtered_votes = years.merge(filtered_votes, on=["Year"], how="left")
    x = years["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            fa=filtered_fa[fa_type],
            fa_labels=filtered_fa[fa_type].apply(lambda x: round(x) if x else x),
            alignment_percentage=filtered_votes["alignment_percentage"],
        )
    )
    labels = LabelSet(
        x="x", y="fa", text="fa_labels", x_offset=-15, y_offset=5, source=source
    )
    plot = figure(width=950, height=600, title="Overall FA vs Alignment")
    plot.add_layout(labels)
    plot.y_range = Range1d(
        start=0, end=filtered_fa[fa_type].max() + filtered_fa[fa_type].max() / 2
    )
    plot.extra_y_ranges = {"percent": Range1d(start=-125, end=125)}
    plot.add_layout(
        LinearAxis(y_range_name="percent", axis_label="Alignment Percentage"),
        "right",
    )

    ap_par = np.polyfit(x, filtered_votes["alignment_percentage"].to_numpy(), 1, full=True)
    ap_slope = ap_par[0][0]
    ap_intercept = ap_par[0][1]
    ap_y_predicted = [ap_slope * i + ap_intercept for i in x]
    plot.line(
        x,
        ap_y_predicted,
        y_range_name="percent",
        color="blue",
        legend_label="y="
        + str(round(ap_slope, 2))
        + "x+"
        + str(round(ap_intercept, 2)),
    )
    plot.circle(
        x=x,
        y=filtered_votes["alignment_percentage"].tolist(),
        y_range_name="percent",
        legend_label="Alignment %",
    )
    plot.vbar(
        x=x,
        top=filtered_fa[fa_type],
        width=0.9,
        fill_color=PuBu[7][2],
        legend_label=f"Financial assistance:{fa_type}",
    )
    plot.xaxis.ticker = list(range(x.min(), x.max()))

    show(plot)

fa_type = get_filter("fa_type")
country_group = get_filter("country_group")
topic = get_filter("topic")
plot2 = widgets.interactive(plot, fa_type=fa_type, country_group=country_group, topic=topic)


In [5]:
# Overall voting alignment with China per vote type/topic, over time for all countries

votes = df_votes

def colour_map(x):
    if x > 0:
        return "green"
    elif x < 0:
        return "red"
    else:
        return "black"


def plot(country_group, topic):
    plot = None
    group_by = {"Year", "Country Group", "Topic"}
    filtered_votes = votes

    if country_group == "All":
        group_by.remove("Country Group")
    else:
        filtered_votes = df_votes[df_votes["Country Group"]== country_group]
    if topic == "All":
        group_by.remove("Topic")
    else:
        filtered_votes = filtered_votes[filtered_votes["Topic"] == topic]
    
    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_votes = add_measures(filtered_votes)
    filtered_votes = filtered_votes.astype({"Year": str,})
    x = filtered_votes["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            top=filtered_votes["alignment_percentage"],
            labels=filtered_votes["alignment_percentage"].apply(lambda x: str(round(x,2))+'%' if x else x),
            y_offset = filtered_votes["alignment_percentage"].apply(lambda x: 25*np.sign(x)),
            color=filtered_votes["alignment_percentage"].apply(colour_map)
        )
    )
    y_range = Range1d(start=-125, end=125)
    plot = figure(x_range=x,y_range=y_range,height=600, width=900, title="Overall Alignment by Topic over time")

    plot.vbar(
        width=0.9,
        fill_color="color",
        source=source
    )
    labels = LabelSet(
        x="x", y="top", text="labels", x_offset=-15, y_offset="y_offset", source=source
    )
    plot.add_layout(labels)
    show(plot)

country_group = get_filter("country_group")
topic = get_filter("topic")
plot3 = widgets.interactive(plot, country_group=country_group, topic=topic)


In [6]:
# Voting alignment with China and financial assistance, over time, per country

import math
fa = df_fa
votes = df_votes
def get_label_percentage(x):
    if math.isnan(x):
        return 0
    return str(round(x,2))+"%"

def get_label(x):
    if math.isnan(x):
        return 0
    return str(round(x))

def get_alignment_color(x):
    if x <0:
        return Reds[7][2]
    return Greens[7][2]

def plot(fa_type, country):
    plot = None
    group_by = {"Year", "Country"}
    filtered_votes = votes
    filtered_fa = fa
    years = pd.DataFrame({"Year": df["Year"].unique()})

    if country == "All":
        group_by.remove("Country")
    else:
        filtered_votes = filtered_votes[filtered_votes["Country"]== country]
        filtered_fa = filtered_fa[filtered_fa["Country"]==country]
    
    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_fa = filtered_fa.groupby(list(set(filtered_fa.columns) & group_by)).agg(
        {
            "FT_ODI": "sum",
            "AEI_FDI": "sum",
            "BU_ODA": "sum",
            "ODI_MOFCOM": "sum",
        }
    ).reset_index()
    filtered_fa = add_measures(filtered_fa)
    filtered_votes = add_measures(filtered_votes)
    filtered_fa = years.merge(filtered_fa, on=["Year"], how="left")
    filtered_votes = years.merge(filtered_votes, on=["Year"], how="left")
    x = years["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            fa=filtered_fa[fa_type],
            fa_labels=filtered_fa[fa_type].apply(get_label),
            alignment_percentage=filtered_votes["alignment_percentage"],
            alignment_color=filtered_votes["alignment_percentage"].apply(get_alignment_color),
            alignment_label = filtered_votes["alignment_percentage"].apply(get_label_percentage)
        )
    )
    labels = LabelSet(
        x="x", y="fa", text="fa_labels", x_offset=-15, y_offset=5, source=source
    )
    labels2 = LabelSet(
        x="x", y="alignment_percentage",y_range_name="percent", text="alignment_label", x_offset=-35, y_offset=10, source=source
    )
    plot = figure(width=950, height=600, title="FA vs Alignment per country")
    plot.add_layout(labels)
    plot.add_layout(labels2)
    plot.y_range = Range1d(
        start=-(filtered_fa[fa_type].max() + filtered_fa[fa_type].max() / 2), end=filtered_fa[fa_type].max() + filtered_fa[fa_type].max() / 2
    )
    plot.extra_y_ranges = {"percent": Range1d(start=-125, end=125)}
    plot.add_layout(
        LinearAxis(y_range_name="percent", axis_label="Alignment Percentage"),
        "right",
    )
    plot.vbar(
        x=x,
        top=filtered_fa[fa_type],
        width=0.9,
        fill_color=PuBu[7][2],
        legend_label=f"Financial assistance:{fa_type}",
    )
    plot.circle(
        x="x",
        y="alignment_percentage",
        size=10,
        fill_color="alignment_color",
        y_range_name="percent",
        legend_label="Alignment %",
        source=source
    )
    plot.xaxis.ticker = list(range(x.min(), x.max()))

    show(plot)

fa_type = get_filter("fa_type")
country = get_filter("country")
plot4 = widgets.interactive(plot, fa_type=fa_type, country=country)


In [14]:
# Country voting alignment with China pre- and post-vaccine donations, per country

vac = df_vac
votes = df_votes
def plot(country):
    plot = None
    group_by = {"Year", "Country"}
    filtered_votes = votes
    filtered_vaccine = df_vac
    years = pd.DataFrame({"Year": df["Year"].unique()})

    if country == "All":
        group_by.remove("Country")
    else:
        filtered_votes = filtered_votes[filtered_votes["Country"]== country]
        filtered_vaccine = filtered_vaccine[filtered_vaccine["Country"]==country]
    
    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_vaccine = filtered_vaccine.groupby("Country").agg({"Year": "first"})
    filtered_votes = years.merge(filtered_votes, on=["Year"], how="left")
    filtered_votes = add_measures(filtered_votes)
    filtered_votes = filtered_votes.merge(filtered_vaccine, on="Country",how="left", suffixes=("","first"))
    filtered_votes["first_donation"] = filtered_votes.apply(lambda x: int(x["Year"]==x["Yearfirst"]), axis=1)
    x = years["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            alignment_percentage=filtered_votes["alignment_percentage"],
            alignment_color=filtered_votes["alignment_percentage"].apply(get_alignment_color),
            alignment_label = filtered_votes["alignment_percentage"].apply(get_label_percentage)
        )
    )
    labels = LabelSet(
        x="x", y="alignment_percentage", text="alignment_label", x_offset=-35, y_offset=15, source=source
    )
    plot = figure(width=950, height=600, title="Alignment pre and post Vaccine Donation")
    plot.add_layout(labels)
    plot.y_range = Range1d(start=-125, end=125)
    first_donation = Span(location=filtered_vaccine["Year"].min(), dimension='height',
                    line_color='#009E73', line_width=5)
    plot.add_layout(first_donation)
    plot.circle(
        x="x",
        y="alignment_percentage",
        size=10,
        fill_color="alignment_color",
        legend_label="Alignment %",
        source=source
    )
    plot.xaxis.ticker = list(range(x.min(), x.max()))

    show(plot)

fa_type = get_filter("fa_type")
country = widgets.Dropdown(options = df_votes.sort_values("Country")["Country"].unique().tolist())
plot5 = widgets.interactive(plot, country=country)

In [ ]:
display(plot1)
display(plot2)
display(plot3)
display(plot4)
display(plot5)
# buttons = []
# out = widgets.Output(layout={'border': '1px solid black'})
# def on_button_clicked(plot):
#     out.clear_output()
#     with out:
#         display(widgets.VBox([widgets.HBox(plot.children[:-1]),plot.children[-1]]))

# def on_map_button_clicked(b):
#     plot = plot1
#     on_button_clicked(plot)
# def ova_per_country_group_and_topic_clicked(b):
#     plot = plot2
#     on_button_clicked(plot)
# def ova_per_topic_clicked(b):
#     plot = plot3
#     on_button_clicked(plot)
# def cva_vs_fa_per_country_clicked(b):
#     plot = plot4
#     on_button_clicked(plot)
    
# def cva_pre_post_clicked(b):
#     plot = plot5
#     on_button_clicked(plot)





# map_button = widgets.Button(description="Map")
# map_button.on_click(on_map_button_clicked)
# cva_pre_post_vaccine = widgets.Button(description="CVA pre and post vaccine donation")
# cva_pre_post_vaccine.on_click(cva_pre_post_clicked)
# cva_vs_fa_per_country = widgets.Button(description="CVA vs. FA per country")
# cva_vs_fa_per_country.on_click(cva_vs_fa_per_country_clicked)
# ova_per_topic = widgets.Button(description="Overall VA per topic")
# ova_per_topic.on_click(ova_per_topic_clicked)
# ova_per_country_group_and_topic = widgets.Button(description="Overall VA Country Group and topic")
# ova_per_country_group_and_topic.on_click(ova_per_country_group_and_topic_clicked)
# buttons.append(map_button)
# buttons.append(ova_per_country_group_and_topic)
# buttons.append(ova_per_topic)
# buttons.append(cva_vs_fa_per_country)
# buttons.append(cva_pre_post_vaccine)

# with out:
#     display(widgets.VBox([widgets.HBox(plot1.children[:-1]),plot1.children[-1]]))

# vbox = widgets.VBox([widgets.HBox([button for button in buttons]), out])
# display(vbox)

In [13]:
# from bokeh.embed import components

# # Assuming plot1, plot2, plot3, plot4, plot5 are your Bokeh plot objects
# plots = [plot1, plot2, plot3, plot4, plot5]
# print([components(plot) for plot in plots])
# plot_scripts_divs = [components(plot) for plot in plots]

# # Generating script and div for each plot
# scripts = [script for script, div in plot_scripts_divs]
# divs = [div for script, div in plot_scripts_divs]

# # HTML template with placeholders for scripts and divs
# html_template = f"""
# <html>
# <head>
#     <link href="https://cdn.bokeh.org/bokeh/release/bokeh-2.3.3.min.css" rel="stylesheet" type="text/css">
#     <script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.3.3.min.js"></script>
#     <!-- Include additional BokehJS libraries if necessary -->
# </head>
# <body>
#     <!-- Buttons -->
#     <button onclick="showPlot(0)">Map</button>
#     <button onclick="showPlot(1)">Overall VA Country Group and Topic</button>
#     <button onclick="showPlot(2)">Overall VA per Topic</button>
#     <button onclick="showPlot(3)">CVA vs. FA per Country</button>
#     <button onclick="showPlot(4)">CVA Pre and Post Vaccine Donation</button>

#     <!-- Plot Div Containers -->
#     <div id="plotContainer">
#         {"".join(divs)}
#     </div>

#     <!-- Bokeh Scripts -->
#     <script>
#         {''.join(scripts)}
        
#         function showPlot(plotIndex) {{
#             const plots = document.querySelectorAll('#plotContainer > div');
#             plots.forEach((plot, index) => {{
#                 if(index === plotIndex) {{
#                     plot.style.display = 'block';
#                 }} else {{
#                     plot.style.display = 'none';
#                 }}
#             }});
#         }}

#         // Initially hide all plots except the first one
#         document.addEventListener('DOMContentLoaded', function() {{
#             showPlot(0);
#         }});
#     </script>
# </body>
# </html>
# """

# # Write the HTML output to a file
# with open('output.html', 'w') as f:
#     f.write(html_template)

ValueError: Input must be a Model, a Document, a Sequence of Models and Document, or a dictionary from string to Model and Document